In [1]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import det

# import system as sys
from system import avg_normal_params
from methods import RK_system, shoot
# from bifan import bifan

In [2]:
def fx(t):
    return (2 - t) * np.exp(2*t) - 2 * np.exp(t)


def fy(t):
    return (t - 1) * np.exp(2*t) + np.exp(t)


def dx(t, y0, params = None):
    x, y = y0
    return x - y + np.exp(t)


def dy(t, y0, params = None):
    x, y = y0
    return x + 3*y


def system(t, y0, params=None):
    return np.array([dx(t, y0, params), dy(t, y0, params)])


def resid(left, right, params = avg_normal_params, xl = 0, yr = 1):
    x0, y0 = left[0], left[1]
    x1, y1 = right[0], right[1]
    r1, r2 = x0 - xl, y1 - yr
    return np.array([r1, r2])

In [4]:
RK_system(0.5, 0, np.array([1.0001, 1.]), system), RK_system(0.5, 0, np.array([1., 1.]), system)

(array([0.25214071, 0.09018562]), array([0.25208553, 0.09020401]))

In [5]:
iters, vt, D = shoot(0, 1, np.array([1., 1.]), system=system, error = resid, eps=10**(-4))

Iteration: 1
yt = [1. 1.]	 Boundaries: [0.25208557 0.090204  ]	[1.2859701 5.913998 ]	Error: [0.25208557 4.91399813]
	 Changing yt	 y0 : [1.0001 1.    ]	 Boundaries: [0.25214076 0.09018554]	[1.2861065 5.9141364]	Error: [0.25214076 4.91413641]
	 Changing yt	 y1 : [1.     1.0001]	 Boundaries: [0.25210395 0.09022241]	[1.2858347 5.9144087]	Error: [0.25210395 4.91440868]
	 Jacobian: [0.55193901 0.18388033 1.38282776 4.10556793]
yt = [1. 1.]
Iteration: 2
yt = [ 0.9347002  -0.17491655]	 Boundaries: [-6.194395e-05 -1.138986e-01]	[2.7940962 1.034618 ]	Error: [-6.19439525e-05  3.46180201e-02]
	 Changing yt	 y0 : [ 0.9348002  -0.17491655]	 Boundaries: [-6.9023722e-06 -1.1391696e-01]	[2.794231  1.0347537]	Error: [-6.90237221e-06  3.47536802e-02]
	 Changing yt	 y1 : [ 0.9347002  -0.17481655]	 Boundaries: [-4.34971480e-05 -1.13880225e-01]	[2.7939599 1.0350255]	Error: [-4.34971480e-05  3.50254774e-02]
	 Jacobian: [0.5504158  0.18446804 1.35660172 4.07457352]
yt = [ 0.9347002  -0.17491655]
Iteration: 3

In [6]:
vt - np.array(fx(0.5), fy(0.5)), np.array([fx(0.5), fy(0.5)])

(array([ 0.15802985, -0.9644901 ], dtype=float32),
 array([0.7799802 , 0.28958036]))

In [6]:
solver = RK_system
yt = np.array([1., 1.])
t0, t, t1 = 0, 0.5, 1
eps = 0.0001


left = solver(t, t0, yt, system)
right = solver(t, t1, yt, system)
err = resid(left, right)

print(err)

J = np.empty((2, 2))
for i in range(len(yt)):
    yi = yt.copy()
    yi[i] += eps

    left = solver(t, t0, yi, system)
    right = solver(t, t1, yi, system)
    res = resid(left, right)

    J[:, i] = (res - err) / eps
    
J

[0.2521 4.914 ]


array([[0., 0.],
       [1., 4.]])

In [8]:
def f1(x, y):
    return y

def f2(x, y):
    return 2*x

def f3(x, y):
    return 3*x**2

def f4(x, y):
    return 5

def system(x, ys, params = avg_normal_params):
    return np.array([f1(x, ys[0]),
                     f2(x, ys[1]), 
                     f3(x, ys[2]),
                     f4(x, ys[3])])

In [9]:
# y_init=np.random.randn(4)
y_init = np.array([np.e, 1, 1, 5])

rl = RK_system(1, 0, y_init, system=system)
rr = RK_system(1, 2, y_init, system=system)

In [10]:
rl

array([1., 0., 0., 0.])

In [9]:
def resid(left, right, params = avg_normal_params, rl = rl, rr = rr):
    return (left - rl) + (right - rr)

In [14]:
shoot(0, 3, y_init, system=system, error=resid, debug=True)

1 39.99999999992909
2 31.999999999985903
3 39.9999999998758
4 39.99999999989356
Converged in 5 iterations

(5, array([ 1.78305183, -0.25      , -6.125     ,  5.        ]), 40.0)

In [16]:
"""
axis_0 - Res, Sq, Ext
axis_1 - vessel number
axis_2 - [min_patalogy, max_patalogy] 
"""

full_ranges = np.array([[[3, 10], [3, 21], [3, 21], [0.0000001, 15], [0.0000001, 15]], 
                        [[78.5, 1590.4], [0.5, 491], [0.5, 3849], [0.5, 1257], [0.0000001, 1591]],
                        [[0.0000001, 0.1], [0.0000001, 0.5], [0.0000001, 0.5], [0.0000001, 0.6], [0.0000001, 0.6]]])

LIN = 10
full_lins = []

for i, dim in enumerate(full_ranges):
    for j, ves in enumerate(dim):
        full_lins.append(np.linspace(ves[0], ves[1], LIN))
        
full_lins = np.array(full_lins).reshape((3, 5, LIN))

In [17]:
BIF_MAPS = np.array([])
pairs = [list(i) for i in (itertools.combinations(range(15), 2))]

for cnt, p in enumerate(pairs):

    dm1, ves1, dm2, ves2 = p[0] // 5, p[0] % 5, p[1] // 5, p[1] % 5
    
    range_1 = full_lins[dm1, ves1]
    range_2 = full_lins[dm2, ves2]
    params = avg_normal_params.copy()
    
    bif_params = []
    
    for p1 in range_1:
        for p2 in range_2:
            params[dm1, ves1] = p1
            params[dm2, ves2] = p2
            bif_params.append(params.copy())
            
    print(f'Sets of parameters: {len(bif_params)}')
    bif_map = bifan(bif_params, debug=True)
    BIF_MAPS = np.append(BIF_MAPS, bif_map)
    break

Sets of parameters: 100
Critical   iters:6   Error = [ 2.20417129e+05 -8.79396706e+06 -8.42255335e+34 -1.35841600e+68
 -1.59736071e+11 -6.59505543e+19  1.59775201e+09  7.12233584e+14
  8.99652911e+07 -2.77233066e+15]1/100   D = 0.0
Critical   iters:6   Error = [-4.37954830e+16 -2.19513138e+32 -8.42255335e+34 -1.35841600e+68
 -1.59736071e+11 -6.59505543e+19  1.59775201e+09  7.12233584e+14
  8.99652911e+07 -2.77233066e+15]2/100   D = 0.0


KeyboardInterrupt: 

In [15]:
(bif_params[0] == bif_params[1]).all()

False

In [21]:
bif_params

[array([[3.0000e+00, 3.0000e+00, 1.2500e+01, 2.0250e+00, 2.0250e+00],
        [5.1055e+02, 9.8150e+01, 6.3185e+02, 1.6690e+02, 1.6690e+02],
        [2.1500e-02, 7.0000e-02, 6.0000e-02, 1.5000e-01, 2.7000e-01]]),
 array([[3.0000e+00, 5.0000e+00, 1.2500e+01, 2.0250e+00, 2.0250e+00],
        [5.1055e+02, 9.8150e+01, 6.3185e+02, 1.6690e+02, 1.6690e+02],
        [2.1500e-02, 7.0000e-02, 6.0000e-02, 1.5000e-01, 2.7000e-01]]),
 array([[3.0000e+00, 7.0000e+00, 1.2500e+01, 2.0250e+00, 2.0250e+00],
        [5.1055e+02, 9.8150e+01, 6.3185e+02, 1.6690e+02, 1.6690e+02],
        [2.1500e-02, 7.0000e-02, 6.0000e-02, 1.5000e-01, 2.7000e-01]]),
 array([[3.0000e+00, 9.0000e+00, 1.2500e+01, 2.0250e+00, 2.0250e+00],
        [5.1055e+02, 9.8150e+01, 6.3185e+02, 1.6690e+02, 1.6690e+02],
        [2.1500e-02, 7.0000e-02, 6.0000e-02, 1.5000e-01, 2.7000e-01]]),
 array([[3.0000e+00, 1.1000e+01, 1.2500e+01, 2.0250e+00, 2.0250e+00],
        [5.1055e+02, 9.8150e+01, 6.3185e+02, 1.6690e+02, 1.6690e+02],
        [2.1

In [ ]:
BIF_MAPS

In [ ]:
im = BIF_MAPS[0]
plt.imshow(im, cmap='gray')